# Capstone proposal by Mirko Salomon 

# BEVERAGE MACHINE CHURN PREDICTION

* ## [1) Introduction and preparation](#Introduction) 

    *   
        #### Data Load
        
        #### Hyperparameters

    
* ## [2) Decision Tree model](#DT)
    
    *   
        #### Train the model
        
        #### Test data with best parameters 
        
        #### Confusion Matrix
        
        #### Tree Plot
        
        #### Prediction of churn

## 1) Introduction and preparation<a class="anchor" id="Introduction"></a>

The idea behind decision tree is to learn a set of if-then-else rules that lead to a final
decision. In classification tasks, this decision is a label.

Based on the flowchart from Scikit Learn I should try it.

Simple to explain and can be ploted.

It can help to understand and works with categorical features

It can predict probability, features that are not really important will not influence a lot the result, multicollinearity is not a problem

I will use graphviz

I will tune the depth of the tree and the minimum sample per split. I will also work with the class weight.
I might use the both gini and entropy to measure the quality of a split and  both strategy for thee split at each node ('best', 'random')

In [1]:
from sklearn.model_selection import GridSearchCV
import numpy as np

import pandas as pd
import os
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import pickle

import xlrd

import datetime as dt
from datetime import datetime

import collections
from collections import Counter

# Import seaborn
import seaborn as sns


In [2]:
#libraries
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline


from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

from sklearn.model_selection import ParameterGrid

from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
from sklearn.tree import export_graphviz
import graphviz

### Data Load

In [4]:
# Load the pickle file
with open('BM_noTickets_preprocess.p', 'rb') as file:
    BM_noTickets_preprocess = pickle.load(file)

In [5]:
# Load the pickle file
with open('X.p', 'rb') as file:
    X = pickle.load(file)

# Load the pickle file
with open('y.p', 'rb') as file:
    y = pickle.load(file)

# Load the pickle file
with open('X_tr.p', 'rb') as file:
    X_tr = pickle.load(file)

# Load the pickle file
with open('y_tr.p', 'rb') as file:
    y_tr = pickle.load(file)

# Load the pickle file
with open('X_val.p', 'rb') as file:
    X_val = pickle.load(file)

# Load the pickle file
with open('y_val.p', 'rb') as file:
    y_val = pickle.load(file)

# Load the pickle file
with open('X_te.p', 'rb') as file:
    X_te = pickle.load(file)

# Load the pickle file
with open('y_te.p', 'rb') as file:
    y_te = pickle.load(file)

In [6]:
BM_noTickets_preprocess.head()

,TA Contract Installation Date,Depreciation Start,TA Contract Start Date,TA Contract End Date,#months of data,Churn,Service Category_Installation,Service Category_Installation.,Service Category_Removal,Service Category_Removal.,...,IP Ownership_Exclusive,IP Ownership_Non-Proprietary,IP Ownership_Propr. Comp.,IP Ownership_Proprietary,Trading Partner_%23-Unknown,Trading Partner_Direct,Trading Partner_EVS,G/R/M TB_GTB (Global),G/R/M TB_MTB (Market),G/R/M TB_RTB (Regional)
0,1445.0,1216.0,1441.0,-99.622242,6.965235,False,0.0,1.0,0.0,0.0,...,0,0,1,0,0,0,1,1,0,0
1,1445.0,1961.0,1441.0,-99.622242,25.955358,False,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,1,0,0
2,1445.0,1216.0,1441.0,-99.622242,25.955358,False,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,1,0,0
3,1445.0,1216.0,1441.0,-99.622242,25.955358,False,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,1,0,0
4,1445.0,1216.0,1441.0,-99.622242,25.955358,False,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,1,0,0


### Hyperparameters

#class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)

---

The function to measure the quality of a split. I will try both
criterion = ['gini', 'entropy']

---

splitter = ['best', 'random']

---

The maximum depth of the tree. I will try some depth.
max_depth = [5, 10, 50, 100, None]

---

The minimum number of samples required to split an internal node. I will try some minimum samples

min_samples_split = [2, 4, 10, 40]


---

Weights associated with classes. I will try both

class_weight = ['balanced', None]

## 2) Decision Tree  model<a class="anchor" id="DT"></a>

### Train the model

In [7]:
# Create Decision tree classifier
pipe = Pipeline([
    ('dt', DecisionTreeClassifier(random_state=0))
])

# Define a set of reasonable values
criterion = ['gini', 'entropy']
#splitter = ['best', 'random']
splitter = ['random']
#max_depth = [5, 10, 50, 100, None]
max_depth = [5, 50, None]
#min_samples_split = [2, 4, 10, 40]
min_samples_split = [4, 40]
class_weight = ['balanced', None]


# Define a grid of values
grid = ParameterGrid({
    'dt__criterion': criterion,
    'dt__splitter': splitter,
    'dt__max_depth': max_depth,
    'dt__min_samples_split': min_samples_split,
    'dt__class_weight': class_weight
})


# Save accuracy on test set
test_scores = []

for params_dict in grid:
    # Set parameters
    pipe.set_params(**params_dict)

    # Fit a Decision Tree classifier
    pipe.fit(X_tr, y_tr)

    # Save accuracy on validation set
    params_dict['accuracy'] = pipe.score(X_val, y_val)
    # Save f1 score on validation set
    # predict test instances
    y_pred = pipe.predict(X_val)
    params_dict['f1_macro'] = metrics.f1_score(y_val, y_pred, average='macro')
    
    # Save result
    test_scores.append(params_dict)

# Create Decision tree classifier
pipe = Pipeline([
    ('dt', DecisionTreeClassifier(random_state=0))
])

# Define a set of reasonable values
criterion = ['gini']
splitter = ['random']
max_depth = [ 100]
min_samples_split = [2, 4]
class_weight = [ None]


# Define a grid of values
grid = ParameterGrid({
    'dt__criterion': criterion,
    'dt__splitter': splitter,
    'dt__max_depth': max_depth,
    'dt__min_samples_split': min_samples_split,
    'dt__class_weight': class_weight
})


# Save accuracy on test set
test_scores = []

for params_dict in grid:
    # Set parameters
    pipe.set_params(**params_dict)

    # Fit a Decision Tree classifier
    pipe.fit(X_tr, y_tr)

    # Save accuracy on validation set
    params_dict['accuracy'] = pipe.score(X_val, y_val)
    # Save f1 score on validation set
    # predict test instances
    y_pred = pipe.predict(X_val)
    params_dict['f1_macro'] = metrics.f1_score(y_val, y_pred, average='macro')
    
    # Save result
    test_scores.append(params_dict)

In [8]:
# Create DataFrame with test scores
scores_df = pd.DataFrame(test_scores)

# Top five scores
scores_df.sort_values(by='f1_macro', ascending=False).head()

,dt__class_weight,dt__criterion,dt__max_depth,dt__min_samples_split,dt__splitter,accuracy,f1_macro
22,None,entropy,NaN,4,random,0.974405,0.971481
20,None,entropy,50.0,4,random,0.974405,0.971481
16,None,gini,NaN,4,random,0.973826,0.970831
8,balanced,entropy,50.0,4,random,0.972438,0.969301
10,balanced,entropy,NaN,4,random,0.972438,0.969301


### Test data with best parameters 

In [9]:
# Best value

params_dict = {'dt__criterion': 'gini', 
               'dt__splitter': 'random', 
               'dt__min_samples_split': 4, 
               'dt__max_depth': None,
               'dt__class_weight': None
              }

# Create Decision tree classifier
pipe = Pipeline([
    ('dt', DecisionTreeClassifier(
        random_state=0
))
])


# Save accuracy on test set
test_scores = []

# Set parameters
pipe.set_params(**params_dict)

# Fit a Decision Tree classifier
pipe.fit(X_tr, y_tr)


# Save accuracy on test set
params_dict['accuracy'] = pipe.score(X_te, y_te)
# Save f1 score on test set
# predict test instances
y_pred = pipe.predict(X_te)
params_dict['f1_macro'] = metrics.f1_score(y_te, y_pred, average='macro')
    
# Save result
test_scores.append(params_dict)
    
# Create DataFrame with test scores
scores_df = pd.DataFrame(test_scores)

# Top five scores
scores_df.sort_values(by='f1_macro', ascending=False)

,dt__criterion,dt__splitter,dt__min_samples_split,dt__max_depth,dt__class_weight,accuracy,f1_macro
0,gini,random,4,None,None,0.972563,0.969375


We have an F1 Macro score of 83.3% and an accuracy of 93.2% with the test data. Which is even better than the validation data results.

In [10]:
# F1_score
dt_F1Macro = params_dict['f1_macro']

# Accuracy
dt_accuracy = params_dict['accuracy']

In [11]:
# Load the npz file for results
with np.load('Results.npz', allow_pickle=False) as npz_file:
    F1Score = npz_file['test_F1Score']
    accuracy = npz_file['test_accuracy']
    models = npz_file['models']
    
# Fill the calculated result value
F1Score[3] = dt_F1Macro
accuracy[3] = dt_accuracy

print('F1_Results:', F1Score)
print('Accuracy:', accuracy)

F1_Results: [0.39715065 0.         0.         0.96937541 0.         0.
 0.         0.        ]
Accuracy: [0.65878923 0.         0.         0.97256318 0.         0.
 0.         0.        ]


In [12]:
# Modify the Numpy array
#Model = models #unchanged
Result = F1Score
Accuracy = accuracy

# Store the changes in the results npz file
np.savez('Results.npz', models = models, test_F1Score = Result,  test_accuracy = Accuracy)

In [13]:
# Check the refreshed results
# Load the npz file for results
with np.load('Results.npz', allow_pickle=False) as npz_file:
    # It's a dictionary-like object    
    print(list(npz_file.keys()))
    # Load the arrays
    print('Models:', npz_file['models'])
    print('F1_Results:', npz_file['test_F1Score'])
    print('Accuracy:', npz_file['test_accuracy'])

['models', 'test_F1Score', 'test_accuracy']
Models: ['Baseline' 'Logistic Regression' 'KNeighbors' 'Decision Tree'
 'Random Forest' 'XGBoost' 'SelectedModel_wTickets'
 'SelectedModel_wTickets&Telemetry']
F1_Results: [0.39715065 0.         0.         0.96937541 0.         0.
 0.         0.        ]
Accuracy: [0.65878923 0.         0.         0.97256318 0.         0.
 0.         0.        ]


### Confusion Matrix

In [14]:
#Classification reports and confusion matrices are commonly used for reporting the performance of classifiers when working with imbalanced datasets.

# Print the confusion matrix
print(metrics.confusion_matrix(y_te, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_te, y_pred, digits=3))

[[23320   403]
 [  585 11702]]
              precision    recall  f1-score   support

       False      0.976     0.983     0.979     23723
        True      0.967     0.952     0.959     12287

    accuracy                          0.973     36010
   macro avg      0.971     0.968     0.969     36010
weighted avg      0.973     0.973     0.973     36010



In [15]:
#Classification reports and confusion matrices are commonly used for reporting the performance of classifiers when working with imbalanced datasets.
ConfMat_df = pd.DataFrame(metrics.confusion_matrix(y_te, y_pred).T, columns=['False Condition', 'True Condition'],index=['Predicted False', 'Predicted True'])
ConfMat_df

,False Condition,True Condition
Predicted False,23320,585
Predicted True,403,11702


We have a good F1 Score for the False Condition but not as good for the True Condition.

In [16]:
# Save the Dataframe into a pickle file
with open('Decision Tree_ConfMat_df.p', 'wb') as file:
    pickle.dump(ConfMat_df, file)

### Tree Plot

In [17]:
dt=DecisionTreeClassifier(
    random_state=0,
    criterion = 'gini',
    splitter = 'best',
    max_depth = 20,
    min_samples_split = 10,
    class_weight = None
).fit(X_tr, y_tr)

I limited the maximal depth to 10 otherwise it would become too big. 

In [18]:
##### Plot the corresponding tree with graphviz

ColumnFeatures = BM_noTickets_preprocess.drop('Churn', axis=1).columns

# Export decision tree
dot_data = export_graphviz(
    dt, out_file=None,
feature_names=ColumnFeatures, 
#class_names=['', ''],
    filled=True, rounded=True, proportion=True
)

# Display decision tree
graphviz.Source(dot_data)

'#months of data', 'Depreciation Start',
       'Industry (EC ID)_0614 InStore Food Service',
       'End Date in Local Time Zone_x', 'Sales Organisation_Pakistan',
       'G/R/M TB_MTB (Market)', 'Position_#', 'Position_LOAN',
       'Incident Category_New Customer / Installation Point',
       'User Status_Installed', 'Model Group_Other',
       'User Status_To be removed', 'Service Category_Installation.',
       'Last_visit_diff_months', 'Model Vendor_SAI Vending',
       'Trading Partner_Direct', 'End Date in Local Time Zone_y',
       'Industry (EC ID)_0614 Convenience OOH',
       'Sales Organisation_Nestlé India', 'System Brands_Nescafé Branded'
        
        
        Industry Code 1 (EC ID)_Not assigned   Incident Category_2.e Hydraulic Cooling/Freezing 
        Account ABC Classification (EC ID)_Not assigned
        Industry (EC ID)_0606 Hotel/Accommodation
        Model_NESCAFE Milano FTS60 E
        Position_LOAN
        Last_visit_diff_months
        Generation_Gen. 2 
        
        
        Pakistan
        IP Ownership_Exclusive 
        three_months_avg 
        Industry Code 1 (EC ID)_060705 Tourist Attraction
        User Status_Installed
        Industry Code 1 (EC ID)_090101 Distribution Center 
        
        UK
        Incident Category_New Customer / Installation Point
        Model_NESCAFE Alegria FTS120 
        Service Category_Installation

Gives an idea of how the model is splitting the data in the decision tree by learning a set of if-then-else rules
For example after the first True Condition, the second rule is whether or not the "System Brands" = "prstzé cppzprp".

### Prediction of churn

Churn prediction from the model

In [20]:
np.info(X)

class:  ndarray
shape:  (180046, 1252)
strides:  (8, 1440368)
itemsize:  8
aligned:  True
contiguous:  False
fortran:  True
data pointer: 0x23341813040
byteorder:  little
byteswap:  False
type: object


In [21]:
# Need the one in the same format, so with removed Sales Org
# Load the pickle file
with open('BeverageMachine_withSerial.p', 'rb') as file:
    BeverageMachine_withSerial = pickle.load(file)

In [22]:
BeverageMachine_withSerial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180046 entries, 0 to 180045
Columns: 1257 entries, Serial ID to G/R/M TB_RTB (Regional)
dtypes: bool(1), float64(146), int32(2), int64(2), object(5), uint8(1101)
memory usage: 402.1+ MB


In [23]:
BM_noTickets_preprocess.columns

Index(['TA Contract Installation Date', 'Depreciation Start',
       'TA Contract Start Date', 'TA Contract End Date', '#months of data',
       'Churn', 'Service Category_Installation',
       'Service Category_Installation.', 'Service Category_Removal',
       'Service Category_Removal.',
       ...
       'IP Ownership_Exclusive', 'IP Ownership_Non-Proprietary',
       'IP Ownership_Propr. Comp.', 'IP Ownership_Proprietary',
       'Trading Partner_%23-Unknown', 'Trading Partner_Direct',
       'Trading Partner_EVS', 'G/R/M TB_GTB (Global)', 'G/R/M TB_MTB (Market)',
       'G/R/M TB_RTB (Regional)'],
      dtype='object', length=1253)

# Load the pickle file
with open('BM_noTickets_preprocess.p', 'rb') as file:
    BM_noTickets_preprocess = pickle.load(file)

In [24]:
X2 = BeverageMachine_withSerial.drop(['Churn', 'Serial ID'], axis=1)

In [25]:
name = ['False', 'True']
No=BeverageMachine_withSerial['Serial ID']
predictions = pipe.predict_proba(X2)
# With two column indices, values same  
# as dictionary keys 
df2 = pd.DataFrame(predictions, index=No ,columns = name) 
df2.head()

ValueError: could not convert string to float: 'Deployed'